# EDA for Game Reviews data

In [4]:
import duckdb
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [8]:
db_conn = duckdb.connect("../data/steam.duckdb")
games_df = db_conn.sql("""
SELECT * FROM game_features QUALIFY ROW_NUMBER() OVER (
    PARTITION BY game_id ORDER BY game_review_day DESC, game_scrape_date DESC
) = 1
""").pl()
games_df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

game_index,game_id,game_name,game_is_free,game_developers,game_publishers,game_categories,game_genres,game_steam_release_date,game_release_date,game_prerelease_date,game_short_description,game_scrape_date,game_review_score,game_review_score_description,game_index_1,game_review_day,game_num_reviews,game_num_positive_reviews,game_num_negative_reviews,game_weighted_score
i64,i64,str,bool,list[str],list[str],list[str],list[str],datetime[μs],"datetime[μs, America/Lima]","datetime[μs, America/Lima]",str,date,i64,str,i64,date,i64,"decimal[38,0]","decimal[38,0]",f64
7750,472680,"""Elisa: The Innkeeper - Prequel""",true,"[""Neoclassic Games""]","[""Neoclassic Games""]","[""Single-player"", ""Steam Achievements""]","[""Adventure"", ""Casual"", … ""Free To Play""]",2016-06-14 00:00:00,2016-06-14 00:00:00 -05,2016-06-14 00:00:00 -05,"""This short Visual novel can be…",2025-08-12,5,"""Mixed""",7750,2025-08-16,176,110,66,0.616516
7783,487220,"""Freebie""",false,"[""Free Games Info"", ""hyperboreanGames""]","[""Back To Basics Gaming""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Casual"", ""Indie""]",2016-06-16 00:00:00,2016-06-16 00:00:00 -05,2016-06-16 00:00:00 -05,"""Freebie is a casual game where…",2025-08-12,6,"""Mostly Positive""",7783,2025-07-18,256,162,94,0.62968
9689,491530,"""Notruf 112 | Emergency Call 11…",false,"[""Crenetic GmbH Studios""]","[""Aerosoft GmbH""]","[""Single-player"", ""Steam Workshop"", … ""Family Sharing""]","[""Action"", ""Casual"", ""Simulation""]",2016-11-09 00:00:00,2016-11-09 00:00:00 -05,2016-11-09 00:00:00 -05,"""Emergency Call 112 – The Fire …",2025-08-12,6,"""Mostly Positive""",9689,2017-02-17,5,2,3,0.402935
8334,497180,"""Street Legal Racing: Redline v…",false,"[""Invictus Games Ltd."", ""ImageCode""]","[""Activision Value Publishing Inc."", ""ImageCode""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Racing"", ""Simulation""]",2016-08-05 00:00:00,2016-08-05 00:00:00 -05,2016-08-05 00:00:00 -05,"""The most popular vehicle mecha…",2025-08-12,8,"""Very Positive""",8334,2016-08-07,1,0,1,0.0
12156,505080,"""Gakuen Club""",false,"[""OperaHouse Corporation""]","[""Dogenzaka Lab""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Adventure""]",2017-05-09 00:00:00,2017-05-09 00:00:00 -05,2017-05-09 00:00:00 -05,"""Gakuen Club is a school life l…",2025-08-12,5,"""Mixed""",12156,2019-12-26,8,6,2,0.74686
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
12192,551050,"""Die With Glory — Point and Cli…",false,"[""Castle""]","[""Veslo Games""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Adventure"", ""Indie""]",2017-05-11 00:00:00,2017-05-11 00:00:00 -05,2017-05-11 00:00:00 -05,"""Die With Glory is a humor-pack…",2025-08-12,4,"""Mostly Negative""",12192,2024-12-10,28,22,6,0.802541
10260,553110,"""Alpine Ski VR""",false,"[""Suchworks Ltd""]","[""Suchworks Ltd""]","[""Single-player"", ""Full controller support"", … ""Family Sharing""]","[""Casual"", ""Racing"", ""Sports""]",2016-12-16 00:00:00,2016-12-16 00:00:00 -05,2016-12-16 00:00:00 -05,"""Alpine Ski VR by Suchworks Stu…",2025-08-12,5,"""Mixed""",10260,2025-06-11,237,128,109,0.538628
9903,553910,"""Mystic Defense""",false,"[""Mystified Games""]","[""Mystified Games""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Action"", ""Casual"", ""Indie""]",2016-11-23 00:00:00,2016-11-23 00:00:00 -05,2016-11-23 00:00:00 -05,"""Take control as a mage and unl…",2025-08-12,0,"""2 user reviews""",9903,2025-07-07,31,28,3,0.901628


In [10]:
games_df.sort("game_num_reviews", descending=True)

game_index,game_id,game_name,game_is_free,game_developers,game_publishers,game_categories,game_genres,game_steam_release_date,game_release_date,game_prerelease_date,game_short_description,game_scrape_date,game_review_score,game_review_score_description,game_index_1,game_review_day,game_num_reviews,game_num_positive_reviews,game_num_negative_reviews,game_weighted_score
i64,i64,str,bool,list[str],list[str],list[str],list[str],datetime[μs],"datetime[μs, America/Lima]","datetime[μs, America/Lima]",str,date,i64,str,i64,date,i64,"decimal[38,0]","decimal[38,0]",f64
11545,578080,"""PUBG: BATTLEGROUNDS""",true,"[""PUBG Corporation""]","[""KRAFTON, Inc.""]","[""Multi-player"", ""PvP"", … ""Remote Play on Tablet""]","[""Action"", ""Adventure"", … ""Free To Play""]",2017-12-21 00:00:00,2017-12-21 00:00:00 -05,2017-03-23 10:25:15 -05,"""PUBG: BATTLEGROUNDS, the high-…",2025-08-12,5,"""Mixed""",11545,2025-08-25,1746069,970793,775276,0.55522
950,92100,"""DETOUR""",false,"[""Geoff 'Zag' Keene"", ""Richard Keene""]","[""Geoff 'Zag' Keene""]","[""Single-player"", ""Multi-player"", … ""Family Sharing""]","[""Indie"", ""Strategy""]",2011-05-16 00:00:00,2011-05-16 00:00:00 -05,2011-05-16 00:00:00 -05,"""A true test your wits and spat…",2025-08-12,5,"""Mixed""",950,2025-08-23,1141224,1114096,27128,0.978078
1959,252490,"""Rust""",false,"[""Facepunch Studios""]","[""Facepunch Studios""]","[""Multi-player"", ""MMO"", … ""Remote Play on Tablet""]","[""Action"", ""Adventure"", … ""RPG""]",2018-02-08 00:00:00,2018-02-08 00:00:00 -05,2013-12-11 13:42:09 -05,"""The only aim in Rust is to sur…",2025-08-12,8,"""Very Positive""",1959,2025-08-21,1040475,910907,129568,0.876089
84634,2934620,"""Dance Eden""",false,"[""DigiEden""]","[""DigiEden""]","[""Single-player"", ""Multi-player"", … ""Family Sharing""]","[""Action"", ""Casual"", … ""Early Access""]",2024-08-19 00:00:00,2024-08-19 00:00:00 -05,2024-08-19 00:00:00 -05,"""Dance Eden is a VR/MR dance ga…",2025-08-12,7,"""Positive""",84634,2025-08-20,852419,822781,29638,0.96651
9202,431960,"""Wallpaper Engine""",false,"[""Wallpaper Engine Team""]","[""Wallpaper Engine Team""]","[""Steam Achievements"", ""Steam Trading Cards"", … ""Includes level editor""]","[""Casual"", ""Indie"", … ""Utilities""]",2018-11-16 00:00:00,2018-11-16 00:00:00 -05,2016-10-10 14:39:08 -05,"""Use stunning live wallpapers o…",2025-08-12,9,"""Overwhelmingly Positive""",9202,2025-08-21,849357,832798,16559,0.980876
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
48113,1783700,"""Gachi Bird""",false,"[""Gachi Master""]","[""Gachi Master Games""]","[""Single-player"", ""Steam Achievements"", ""Family Sharing""]","[""Action"", ""Adventure"", … ""Indie""]",2021-10-28 00:00:00,2021-10-28 00:00:00 -05,2021-10-28 00:00:00 -05,"""♂️Gachi Bird♂️ - play the mast…",2025-08-12,0,"""8 user reviews""",48113,2022-08-05,1,1,0,1.0
50542,1837440,"""Pyramid Quest""",false,"[""EntwicklerX""]","[""EntwicklerX""]","[""Single-player"", ""Steam Achievements"", … ""Family Sharing""]","[""Adventure"", ""Indie""]",2022-01-20 00:00:00,2022-01-20 00:00:00 -05,2022-01-20 00:00:00 -05,"""Pyramid Quest is an exploratio…",2025-08-12,0,"""7 user reviews""",50542,2022-01-27,1,1,0,1.0
8208,489810,"""BlockAid""",false,"[""SEGA Hardlight™""]","[""War Child Gaming""]","[""Single-player"", ""Family Sharing""]","[""Casual"", ""Indie"", ""Strategy""]",2016-07-26 00:00:00,2016-07-26 00:00:00 -05,2016-07-26 00:00:00 -05,"""Provide safe passage to a neve…",2025-08-12,0,"""6 user reviews""",8208,2020-05-09,1,1,0,1.0


In [11]:
db_conn.close()

## TODO: EDA
* Number of reviews per game
* Correlation between months since release and number of reviews (positive / negative)
*
* Correlation between is_free and number of reviews (see Welch t-test, Mann-Whitney U, permutation test)
* Game developers and publishers for similar games
* Way to find similarity between games
* Analysis of game description